# Requirements

In [113]:
!pip install javalang

In [1]:
import os
import hashlib
import ast       #Library to produce AST and analyze it
import astor     #Library to produce the actual code of an AST
import pandas as pd
#import javalang
#from javalang.tree import *

In [2]:
hash_dict = {}
filesname = []
file_directories = []

In [3]:
"""
In this part we make a list of the files' name and their own directory in the given directory.
A dictionary is also produced having a hash value of the files name as the key and their actual name as the value.
"""

directory = "/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        filesname.append(filename)
        file_directories.append(f)

In [13]:
file_directories

['/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/oauth.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/timeline.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/basetest_project_root.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/repository.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/test_feeds.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/taskd.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/__init__.py',
 '/home/amirmahdi/projects/Ada

# Extracting function blocks from source code

In [4]:
"""
This function accepts the source code as a string. It first parse it to AST, then iterate through AST and selects 
nodes that are representing function definition (FunctionDef)
"""

def extract_functions(source_code):
    tree = ast.parse(source_code)
    functions = []
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            functions.append(node)
    return functions

In [62]:
"""
Creating the func_of_files list which each of its elements it is a list of function nodes of a file in the
directory.
These nodes (subtree of the AST) could be used to creat the actual functions in string by using following code:

source_code = astor.to_source(node)

Also, in this an ID is assigned to each function (based on its name) by using hashlib, then this ID is used as 
the key value of the "hash_dict" which holds a list containing function's name and its directory as the 
dictionary's value.
"""

func_nodes_of_files = []
for file_dir in file_directories:
    with open(file_dir, "r") as file:
        source_code = file.read()
    functions = extract_functions(source_code)
    for function in functions:
        functions_copy = functions.copy() # Make a copy of list of functions found in current file
        functions_copy.remove(function)    #creating a list of neighbours of current function (in itteratin) by removing itself from functions_copy
        hash_dict[ hashlib.sha1(str.encode(function.name)).hexdigest() ] = [file_dir.split("/")[-1], file_dir, 
                                                                           functions_copy] 
    func_nodes_of_files.extend(functions)

In [63]:
func_nodes_of_files[0].name

'dummy_oauths'

In [7]:
source = astor.to_source(func_nodes_of_files[0])
print(source)

def dummy_oauths():
    from allura.controllers.rest import Oauth1Validator
    dummy_cons_tok = OAuthConsumerToken(api_key=Oauth1Validator().
        dummy_client, name='dummy client, for oauthlib implementation',
        user_id=None)
    session(dummy_cons_tok).flush(dummy_cons_tok)
    dummy_req_tok = OAuthRequestToken(api_key=Oauth1Validator().
        dummy_request_token, user_id=None, validation_pin='dummy-pin')
    session(dummy_req_tok).flush(dummy_req_tok)
    dummy_access_tok = OAuthAccessToken(api_key=Oauth1Validator().
        dummy_access_token, user_id=None)
    session(dummy_access_tok).flush(dummy_access_tok)



# Structring Dataset by extracting functions' features

### 1. Input-dependent loops

In [79]:
def loop_finder(function_node):
    loops = []
    for node in ast.walk(function_node):
        if isinstance(node, ast.For) or isinstance(node, ast.While):
            loops.append(node)
    return loops

In [108]:
def keyword_finder(source_code):
    tree = ast.parse(source_code)
    functions = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            functions.append(node)
    return functions

In [149]:
def keyword_finder(source_code):
    tree = ast.parse(source_code)
    keywords = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Assign):
            for node_child in ast.walk(node):
                if isinstance(node_child, ast.Call):
                    #print(astor.to_source(node))
                    keywords.extend(astor.to_source(node).split("=")[0].split(","))
                    #keywords.extend([var.id for var in node.targets])
    return keywords

In [150]:
functions = []
for file_dir in file_directories:
    with open(file_dir, "r") as file:
        source_code = file.read()
    functions.extend(keyword_finder(source_code))
print(len(functions))
print(astor.to_source(functions[10]))
print([f.id for f in functions[10].targets])

log = logging.getLogger(__name__)

_id = FieldProperty(S.ObjectId)

type = FieldProperty(str)

api_key = FieldProperty(str, if_missing=lambda : h.nonce(20))

api_key = FieldProperty(str, if_missing=lambda : h.nonce(20))

secret_key = FieldProperty(str, if_missing=h.cryptographic_nonce)

last_access = FieldProperty(datetime)

type = FieldProperty(str, if_missing='consumer')

name = FieldProperty(str)

description = FieldProperty(str, if_missing='')

description_cache = FieldProperty(MarkdownCache)

user = RelationProperty('User')

type = FieldProperty(str, if_missing='request')

consumer_token_id = ForeignIdProperty('OAuthConsumerToken')

callback = FieldProperty(str)

validation_pin = FieldProperty(str)

type = FieldProperty(str, if_missing='access')

consumer_token_id = ForeignIdProperty('OAuthConsumerToken')

request_token_id = ForeignIdProperty('OAuthToken')

is_bearer = FieldProperty(bool, if_missing=False)

user = RelationProperty('User')

consumer_token = RelationProperty('OAuthC

status = getattr(self.artifact, 'status', None)

filename = request.path.rsplit('/', 1)[-1]

log = logging.getLogger(__name__)

user = M.User.by_username(user)

pname = unquote(name)

pname = six.ensure_text(pname)

provider = plugin.ProjectRegistrationProvider.get()

self.oauth = OAuthNegotiator()

self.auth = AuthRestController()

summary = dict()

stats = dict()

r = g.theme._get_site_notification(url=url, user=c.user, tool_name=
    tool_name, site_notification_cookie_value=cookie)

neighborhood = M.Neighborhood.query.get(url_prefix='/' + name + '/')

consumer_token = M.OAuthConsumerToken.query.get(api_key=request.client_key)

req_token = M.OAuthRequestToken(api_key=token['oauth_token'], secret_key=
    token['oauth_token_secret'], consumer_token_id=consumer_token._id,
    callback=request.oauth_params.get('oauth_callback', 'oob'))

req_token = M.OAuthRequestToken(api_key=token['oauth_token'], secret_key=
    token['oauth_token_secret'], consumer_token_id=consumer_token._id,
    ca

projects = pq.skip(start).limit(int(limit)).all()

projects = pq.skip(start).limit(int(limit)).all()

projects = pq.skip(start).limit(int(limit)).all()

categories = M.ProjectCategory.query.find({'parent_id': None}).sort('name'
    ).all()

categories = M.ProjectCategory.query.find({'parent_id': None}).sort('name'
    ).all()

categories = M.ProjectCategory.query.find({'parent_id': None}).sort('name'
    ).all()

c.custom_sidebar_menu = c.custom_sidebar_menu + [SitemapEntry(cat.label, 
    self.neighborhood.url() + 'browse/' + cat.name) for cat in categories]

c.custom_sidebar_menu = c.custom_sidebar_menu + [SitemapEntry(cat.label, 
    self.neighborhood.url() + 'browse/' + cat.name) for cat in categories]

p = plugin.ProjectRegistrationProvider.get()

result = p.verify_phone(c.user, number)

request_id = result.pop('request_id', None)

p = plugin.ProjectRegistrationProvider.get()

request_id = session.get('phone_verification.request_id')

number_hash = session.get('phone_verification.

author = FieldProperty(dict(id=S.ObjectId, username=str, display_name=str,
    logged_ip=str))

timestamp = FieldProperty(datetime)

data = FieldProperty(None)

version = FieldProperty(S.Int, if_missing=0)

_id = FieldProperty(str, if_missing=h.gen_message_id)

slug = FieldProperty(str, if_missing=h.nonce)

full_slug = FieldProperty(str, if_missing=None)

parent_id = FieldProperty(str)

app_id = FieldProperty(S.ObjectId, if_missing=lambda : c.app.config._id)

timestamp = FieldProperty(datetime, if_missing=datetime.utcnow)

author_id = FieldProperty(S.ObjectId, if_missing=lambda : c.user._id)

text = FieldProperty(str, if_missing='')

award_id = FieldProperty(S.ObjectId)

_id = FieldProperty(S.ObjectId)

created_by_neighborhood_id = ForeignIdProperty(Neighborhood, if_missing=None)

created_by_neighborhood = RelationProperty(Neighborhood, via=
    'created_by_neighborhood_id')

short = FieldProperty(str, if_missing=h.nonce)

timestamp = FieldProperty(datetime, if_missing=datetime.utcnow)

log = logging.getLogger(__name__)

page_list = ffw.PageList()

page_size = ffw.PageSize()

audit = AuditLog()

delete_form_validators = dict(projects=v.UnicodeString(if_empty=None),
    reason=v.UnicodeString(if_empty=None), disable_users=validators.
    StringBool(if_empty=False))

delete_form_validators = dict(projects=v.UnicodeString(if_empty=None),
    reason=v.UnicodeString(if_empty=None), disable_users=validators.
    StringBool(if_empty=False))

delete_form_validators = dict(projects=v.UnicodeString(if_empty=None),
    reason=v.UnicodeString(if_empty=None), disable_users=validators.
    StringBool(if_empty=False))

delete_form_validators = dict(projects=v.UnicodeString(if_empty=None),
    reason=v.UnicodeString(if_empty=None), disable_users=validators.
    StringBool(if_empty=False))

self.task_manager = TaskManagerController()

self.user = AdminUserDetailsController()

self.delete_projects = DeleteProjectsController()

self.site_notifications = SiteNotificationController()

c.s

seen = set()

types = [el.messagetype for el in self.lastmonth.messages if el.messagetype
     not in seen and not seen.add(el.messagetype)]

a = len([el for el in self.lastmonth.assignedtickets if category in el.
    categories + [None]])

r = len([el for el in self.lastmonth.revokedtickets if category in el.
    categories + [None]])

s, time = reduce(addtuple, [(1, el.solvingtime) for el in self.lastmonth.
    solvedtickets if category in el.categories + [None]], (0, 0))

seen = set()

catlist = [el.category for el in self.general if el.category not in seen and
    not seen.add(el.category)]

now = datetime.utcnow()

s_time = int((close_datetime - open_datetime).total_seconds())

s_time = int((close_datetime - open_datetime).total_seconds())

i = getElementIndex(entry.commits, language=None)

by_cat[cat] = dict(number=n, lines=lines)

by_cat[cat] = dict(assigned=a, solved=s, revoked=r, averagesolvingtime=
    _convertTimeDiff(average))

by_cat[cat] = dict(assigned=a, solved=s, revok

AttributeError: No defined handler for node of type str

In [80]:
df = pd.DataFrame(columns = ['ID', 'number_of_loops'])
#tmp =0
for function_node in func_nodes_of_files:
    found_loops = loop_finder(function_node)
    #tmp += len(found_loops)
    ID = hashlib.sha1(str.encode(function_node.name)).hexdigest()
    number_of_loops = len(found_loops)
    
    new_row = pd.Series({"ID":ID, "number_of_loops":number_of_loops})
    df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
df = df.set_index("ID")
df
#print(tmp)

180


,number_of_loops
ID,
ee2c971f20132061d7e749267b166730ff734a7c,0
d4f01649de54f7e70c368da6571ac5ccda9ddf79,0
bae8ac45f183613712be11aee11ec93b6e36e0d0,0
9aa1369d290e7732b7ca8db743f27d10c0ace093,0
9aa1369d290e7732b7ca8db743f27d10c0ace093,0
...,...
4553a4f2a833f635e06b21120fd95dbd7dc3fe09,0
05bb76f1e435db71754a65f9e8acec7dfdd4c65d,0
4fb170389e1b482ed3c7fae2967caca7629c84a6,0


In [82]:
for function_node in func_nodes_of_files:
    input_dependent_level = 0
    found_loops = loop_finder(function_node)
    if len(found_loops) != 0:
        ID = hashlib.sha1(str.encode(function_node.name)).hexdigest()
        neighbour_functions_name = hash_dict[ID][2]
        for loop in found_loops:
            for neighbour_function in neighbour_functions_name:
                if neighbour_function.name in astor.to_source(loop).split("\n")[0]:
                    input_dependent_level += 1
    df.at[ID, "loop_input_dependent_level"] = int(input_dependent_level)
df["loop_input_dependent_level"] = df["loop_input_dependent_level"].fillna(0)
df

,number_of_loops,loop_input_dependent_level
ID,,
ee2c971f20132061d7e749267b166730ff734a7c,0,0.0
d4f01649de54f7e70c368da6571ac5ccda9ddf79,0,0.0
bae8ac45f183613712be11aee11ec93b6e36e0d0,0,0.0
9aa1369d290e7732b7ca8db743f27d10c0ace093,0,0.0
9aa1369d290e7732b7ca8db743f27d10c0ace093,0,0.0
...,...,...
4553a4f2a833f635e06b21120fd95dbd7dc3fe09,0,0.0
05bb76f1e435db71754a65f9e8acec7dfdd4c65d,0,0.0
4fb170389e1b482ed3c7fae2967caca7629c84a6,0,0.0


In [45]:
fors = [] #this should be dataframe instead
for function_node in func_nodes_of_files:
    found_fors = loop_finder(function_node)
    if len(found_fors) != 0:
        fors.extend(found_fors)
print(astor.to_source(for_nodes_of_files[0][0]))

for node in ([obj, target] + (related_nodes or [])):
    if isinstance(node, Project):
        create_timelines.post(node.node_id)

